# DATASCI W261: Machine Learning at Scale

# This notebook shows a Hadoop MapReduce job of WordCount.

## Data

In [10]:
%%writefile wordcount.txt
hello hi hi hallo
bonjour hola hi ciao
nihao konnichiwa ola
hola nihao hello

Overwriting wordcount.txt


## Mapper

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Writing mapper.py


## Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Writing reducer.py


In [19]:
!chmod +x mapper.py
!chmod +x reducer.py

## Test Code

In [4]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort -k1,1 | python reducer.py

bar	1
foo	3
labs	1
quux	2


## Run wordcount in hadoop

### start yarn and hdfs

In [5]:
!/usr/local/Cellar/hadoop/2.6.0/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.6.0/sbin/start-dfs.sh

/bin/sh: /usr/local/Cellar/hadoop/2.6.0/sbin/start-yarn.sh: No such file or directory
/bin/sh: /usr/local/Cellar/hadoop/2.6.0/sbin/start-dfs.sh: No such file or directory


### create a folder 

In [6]:
!hdfs dfs -mkdir -p /user/miki/week02example

### upload wordcount.txt to hdfs

In [7]:
!hdfs dfs -put wordcount.txt /user/miki/week02example

### Hadoop streaming command

<pre>
hadoop jar hadoopstreamingjarfile \
    -D stream.num.map.output.key.fields=n \
    -mapper mapperfile \
    -reducer reducerfile \
    -input inputfile \
    -output outputfile</pre>

hadoop streaming jar file can be found in your hadoop folder or downloaded from
http://mvnrepository.com/artifact/org.apache.hadoop/hadoop-streaming/2.6.0

In [23]:
!hadoop jar hadoop-streaming-2.7.1.jar -mapper /home/cloudera/Documents/W261-Fall2016/Week02/Example/mapper.py -reducer /home/cloudera/Documents/W261-Fall2016/Week02/Example/reducer.py -input /user/miki/week02example/wordcount.txt -output /user/miki/week02example/wordcountOutput

packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.5.0.jar] /tmp/streamjob5598277463891770878.jar tmpDir=null
16/01/21 13:36:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/21 13:36:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/21 13:36:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/21 13:36:51 INFO mapreduce.JobSubmitter: number of splits:2
16/01/21 13:36:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453405632837_0010
16/01/21 13:36:52 INFO impl.YarnClientImpl: Submitted application application_1453405632837_0010
16/01/21 13:36:52 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1453405632837_0010/
16/01/21 13:36:52 INFO mapreduce.Job: Running job: job_1453405632837_0010
16/01/21 13:36:59 INFO mapreduce.Job: Job job_1453405632837_0010 running in uber mode : false
16/01/21 13:36:59 INFO mapreduce.Job:  map 0% reduce 0%
16/01/21 13:37

### show the results

In [21]:
!hdfs dfs -cat /user/miki/week02example/wordcountOutput/part-00000

bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


### stop yarn and hdfs

In [9]:
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
15/08/21 06:28:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
15/08/21 06:29:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
